<a href="https://colab.research.google.com/github/anirbanmukherjee2709/tsai_end2.0_Session_1/blob/main/Session_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
torch.manual_seed(2)

In [2]:
X = torch.Tensor([[0,0], [0,1], [1,0], [1,1]])
Y = torch.Tensor([0, 1, 1, 0]).view(-1,1)

In [3]:
class XOR(nn.Module):
    def __init__(self, input_dim= 2, output_dim= 1):
        super(XOR, self).__init__()
        self.lin1 = nn.Linear(input_dim, 5)
        self.lin2 = nn.Linear(5, output_dim)
    
    def forward(self, x):
        x = self.lin1(x)
        x = F.tanh(x)
        x = self.lin2(x)
        # x = F.tanh(x) # removing the last activation
        return x

In [4]:
model = XOR()
print(model)
from torchsummary import summary
summary(model, (2, 2))

XOR(
  (lin1): Linear(in_features=2, out_features=5, bias=True)
  (lin2): Linear(in_features=5, out_features=1, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 2, 5]              15
            Linear-2                 [-1, 2, 1]               6
Total params: 21
Trainable params: 21
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


There are no possible combinations to have 44 parameters with 2 input neurons and 1 output neuron and only 1 single hidden layer

In [5]:
display(' 1 hidden layer With Bias')
display([f'in: {i}, hidden_1: {j}, out: {k}' for i in range(1, 10) for j in range(1, 10) for k in range(1, 10) if j*(1+i) + k*(1+j) == 44 and k < i and k < j])

display(' 1 hidden layer Without Bias')
display([f'in: {i}, hidden_1: {j}, out: {k}' for i in range(1, 10) for j in range(1, 10) for k in range(1, 10) if j*i + k*j == 44 and k < i and k < j])

' 1 hidden layer With Bias'

['in: 3, hidden_1: 7, out: 2', 'in: 4, hidden_1: 6, out: 2']

' 1 hidden layer Without Bias'

['in: 8, hidden_1: 4, out: 3', 'in: 9, hidden_1: 4, out: 2']

Trying with 2 hidden layers

In [6]:
display(' 2 hidden layer')
display([f'in: {2}, hidden_1: {j}, hidden_2: {k}, Out: {1}' for j in range(1, 10) for k in range(1, 10) if j*(1+2) + k*(1+j) + 1*(1+k) == 44])

' 2 hidden layer'

['in: 2, hidden_1: 5, hidden_2: 4, Out: 1']

There is only one possible combinations to have 44 parameters with 2 input neurons and 1 output neuron and only 2 hidden layers. Hence, we implement the same

In [7]:
class XOR(nn.Module):
    def __init__(self, input_dim= 2, output_dim= 1):
        super(XOR, self).__init__()
        self.lin1 = nn.Linear(input_dim, 5)
        self.lin2 = nn.Linear(5, 4)
        self.lin3 = nn.Linear(4, output_dim)
    
    def forward(self, x):
        x = self.lin1(x)
        x = torch.tanh(x)
        x = self.lin2(x)
        x = torch.tanh(x)
        x = self.lin3(x)
        x = torch.tanh(x)
        return x

In [8]:
model = XOR()
print(model)
from torchsummary import summary
summary(model, (2, 2))

XOR(
  (lin1): Linear(in_features=2, out_features=5, bias=True)
  (lin2): Linear(in_features=5, out_features=4, bias=True)
  (lin3): Linear(in_features=4, out_features=1, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 2, 5]              15
            Linear-2                 [-1, 2, 4]              24
            Linear-3                 [-1, 2, 1]               5
Total params: 44
Trainable params: 44
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [9]:
def weights_init(model):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            # initialize the weight tensor, here we use a normal distribution
            m.weight.data.normal_(0, 1)

weights_init(model)

In [10]:
loss_func = nn.L1Loss()

In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)

In [12]:
epochs = 2001
steps = X.size(0)
for i in range(epochs):
    for j in range(steps):
        data_point = np.random.randint(X.size(0))
        x_var = Variable(X[data_point], requires_grad=False)
        y_var = Variable(Y[data_point], requires_grad=False)
        
        optimizer.zero_grad()
        y_hat = model(x_var)
        loss = loss_func.forward(y_hat, y_var)
        loss.backward()
        optimizer.step()
        
    if i % 50 == 0:
        print( "Epoch: {0}, Loss: {1}, ".format(i + 1, loss.data.numpy()))

Epoch: 1, Loss: 0.9286401271820068, 
Epoch: 51, Loss: 0.2629197835922241, 
Epoch: 101, Loss: 0.00139617919921875, 
Epoch: 151, Loss: 0.0008918642997741699, 
Epoch: 201, Loss: 0.0014001727104187012, 
Epoch: 251, Loss: 0.999971330165863, 
Epoch: 301, Loss: 0.9999569654464722, 
Epoch: 351, Loss: 0.0008957982063293457, 
Epoch: 401, Loss: 0.3382705748081207, 
Epoch: 451, Loss: 0.00023996829986572266, 
Epoch: 501, Loss: 0.7652166485786438, 
Epoch: 551, Loss: 0.9999520182609558, 
Epoch: 601, Loss: 0.5060203671455383, 
Epoch: 651, Loss: 0.000652611255645752, 
Epoch: 701, Loss: 0.00019043684005737305, 
Epoch: 751, Loss: 0.9999356269836426, 
Epoch: 801, Loss: 0.00036025047302246094, 
Epoch: 851, Loss: 0.9999257922172546, 
Epoch: 901, Loss: 0.0002473592758178711, 
Epoch: 951, Loss: 0.18486538529396057, 
Epoch: 1001, Loss: 0.5732091665267944, 
Epoch: 1051, Loss: 0.5184032917022705, 
Epoch: 1101, Loss: 0.9999232292175293, 
Epoch: 1151, Loss: 0.0003019571304321289, 
Epoch: 1201, Loss: 0.000167429447